In [1]:
# increase Juypter notebook width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))

# Introduction to Python property

By [GreatBahram](https://virgool.io/@greatbahram)

* I usually write about python at https://virgool.io/@GreatBahram/

![Our Society Logo](static/descriptors.png)

### Things to bear in mind

* Please, feel free to **shout out** if you have any question.


* This is not a normal presenation, it's going to be **an interactive tutorial**.

# Python Descriptors

So, **What is** descriptors, **How** can we define one of them?

1. Store the data inside the **descriptor**

```python
class NonNegative:
    """
     A descriptor that forbids negative values.
    """
    def __init__(self, initial):
        self.value = initial
        
    def __get__(self, instance, owner):
        return self.value
    
    def __set__(self, instance, value):
        if value < 0:
            raise ValueError(f"Negative value not allowed: {value}")
        self.value = value
```

2. Store the data inside the **descriptor**

```python
class NonNegative:
    """
     A descriptor that forbids negative values.
    """
    self.STORAGE = {}
    def __init__(self, initial):
        self.intial = initial
        
    def __get__(self, instance, owner):
        # we get here when someone calls x.d, and d is a NonNegative instance
        # instance = x
        # owner = type(x)
        return self.STORAGE.get(instance, self.default)
    
    def __set__(self, instance, value):
        if value < 0:
            raise ValueError("Negative value not allowed: %s" % value)
        self.STORAGE[instance] = value
```

3. Store the data inside the **descriptor**

```python
class NonNegative:
    """
     A descriptor that forbids negative values.
     Store the data into the instance itself.
    """
    def __init__(self, initial):
        self.initial = initial
        
    def __get__(self, instance, owner):
        getattr(instance, '_value', self.initial)
    
    def __set__(self, instance, value):
        if value < 0:
            raise ValueError("Negative value not allowed: %s" % value)
        setattr(instance, '_value, value)
```